In [2]:
import pandas as pd
import numpy as np
import random
import math

In [3]:
X = np.array([
    [1,1,4,0,0], #Dog
    [1,1,4,0,0], #Cat
    [0,1,2,1,0], #Bird
    [0,0,0,0,1], #Fish
    [1,1,4,0,0], #Horse
    [0,0,4,0,1]  #Frog
])

In [4]:
Y = np.array([
    [1], #Dog
    [2], #Cat
    [3], #Bird
    [4], #Fish
    [5], #Horse
    [6]  #Frog
])

In [19]:
class Value:
    def __init__(self, data, _children = (), _op = ''):
        self.data = data
        self.grad = 0.0
        self._prev = _children
        self._op = _op
        self._backward = lambda: None
    
    def __repr__(self, other):
        return f"Value(data = {self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _children = (self,other), _op = '+')

        def backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = backward

        return out
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _children = (self,other), _op = '*')

        def backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = backward

        return out
    
    def __rmul__(self, other):
        return self * other

    def __pow__(self, other):
        assert isinstance(other,[int,float]), "Only int/float input powers are allowed"
        out = Value(self.data ** other, _children = (self), _op = f"** {other}")

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward

    def __truediv__(self):
        return self * (other ** -1)
    
    def __tanh__(self):
        val = (np.exp(2*self.data)-1)/(np.exp(2*self.data)+1)
        out = Value(t,_children= (self), _op = 'tanh')

        def backward():
            self.grad += (1- t**2) * out.grad

        out._backward = backward

    def __relu__():
        out = Value(self.data if self.data > 0 else 0, (self,), 'Relu')

        def backward():
            self.grad += (out.data > 0) * out.data
        out._backward = backward
    
    def softmax(inputs):
        results = []
        exps = sum(np.exp(input.data) for input in inputs)
    
        for input in inputs:
            results.append(Value(np.exp(input.data) / exps))
        return results

    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                seq_list.append(v)
        
        build_seq(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward(v)

In [18]:
class Neuron:
    def __init__(self,nin):
        self.weights = [Value(np.random.randn()) for x in range(nin)]
        self.bias = Value(0.0)

    def __call__(self, x):
        activation = sum((wi *xi for wi,xi in zip(self.weights,x)), self.bias)
        return activation.tanh()
    
    def parameters(self):
        return [self.weights + self.bias]

class Layer:
  
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
  
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
  
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class FeedForward:
  
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
    def __call__(self, x):
        for layer in self.layers:
          x = layer(x)
        return x
  
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]